In [83]:
import random
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
#numpy.set_printoptions(threshold=numpy.nan)
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [84]:
cifar10_name = {
    0: "airplane",
    1: "automobile",
    2: "bird",
    3: "cat",
    4: "deer",
    5: "dog",
    6: "frog",
    7: "horse",
    8: "ship",
    9: "truck"
}

In [85]:
def weight_variable(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.1))
def bias_variable(shape):
    return tf.Variable(tf.constant(0.1, shape=shape))

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[i] for i in idx]
    labels_shuffle = [labels[i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [86]:
data1 = unpickle("cifar-10-batches-py/data_batch_1")
data2 = unpickle("cifar-10-batches-py/data_batch_2")
data3 = unpickle("cifar-10-batches-py/data_batch_3")
data4 = unpickle("cifar-10-batches-py/data_batch_4")
data5 = unpickle("cifar-10-batches-py/data_batch_5")
data = {**data1, **data2, **data3, **data4, **data5}

test = unpickle("cifar-10-batches-py/test_batch")
lables = unpickle("cifar-10-batches-py/batches.meta")[b'label_names']

x = tf.placeholder(tf.float32, [None, 3072])

x_image = tf.reshape(x, [-1, 32, 32, 3])
W_conv1 = weight_variable([5, 5, 3, 32])
b_conv1 = bias_variable([32])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([8 * 8 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 8*8*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)


W_fc2 = weight_variable([1024, 64])
b_fc2 = bias_variable([64])

h_fc2 = tf.matmul(h_fc1, W_fc2) + b_fc2

keep_prob = tf.placeholder(tf.float32)
h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

W_fc3 = weight_variable([64, 10])
b_fc3 = bias_variable([10])

y = tf.matmul(h_fc2_drop, W_fc3) + b_fc3

result = tf.nn.softmax(y)

y_ = tf.placeholder(tf.float32, [None, 10])
learningrate = tf.placeholder(tf.float32, [])
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
train_step = tf.train.AdamOptimizer(learning_rate=learningrate).minimize(cross_entropy)

In [87]:
train_data = []
train_labels = []
for i in range(len(data[b'labels'])):
    img_dat = data[b'data'][i]
    label_dat = np.zeros(10)
    label_dat[data[b'labels'][i]] = 1
    
    train_data.append(img_dat)
    train_labels.append(label_dat)
    
    
test_data = []
test_labels = []
for i in range(len(test[b'labels'])):
    img_dat = data[b'data'][i]
    label_dat = np.zeros(10)
    label_dat[test[b'labels'][i]] = 1
    
    test_data.append(img_dat)
    test_labels.append(label_dat)
    

In [88]:
init = tf.global_variables_initializer()

sess = tf.InteractiveSession()
sess.run(init)

saver = tf.train.Saver()

In [89]:
#Load the model
saver.restore(sess, "./tf-models/cifar10.ckpt")

NotFoundError: Key beta2_power_8 not found in checkpoint
	 [[Node: save_9/RestoreV2_287 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/cpu:0"](_recv_save_9/Const_0, save_9/RestoreV2_287/tensor_names, save_9/RestoreV2_287/shape_and_slices)]]

Caused by op 'save_9/RestoreV2_287', defined at:
  File "c:\users\bibows\appdata\local\programs\python\python35\lib\runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\bibows\appdata\local\programs\python\python35\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\bibows\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\users\bibows\appdata\local\programs\python\python35\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "c:\users\bibows\appdata\local\programs\python\python35\lib\site-packages\ipykernel\kernelapp.py

In [90]:
try:    
    while True:
        test_batch =next_batch(25, test_data, test_labels)
        train_batch = next_batch(25, train_data, train_labels)
        te = cross_entropy.eval(feed_dict={x: test_batch[0], y_: test_batch[1], keep_prob: 1.0})
        tee = cross_entropy.eval(feed_dict={x: train_batch[0], y_: train_batch[1], keep_prob: 1.0})
        print("\rtest error %g, train error %g"%(te, tee), end="")
        for i in range(11):
            batch = next_batch(200, train_data, train_labels)
            sess.run(train_step, feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.3, learningrate: 3e-6})
except KeyboardInterrupt:
    print("")
test_batch =next_batch(200, test_data, test_labels)
train_batch = next_batch(200, train_data, train_labels)
te = cross_entropy.eval(feed_dict={x: test_batch[0], y_: test_batch[1], keep_prob: 1.0})
tee = cross_entropy.eval(feed_dict={x: train_batch[0], y_: train_batch[1], keep_prob: 1.0})
print("test error %g, train error %g"%(te, tee), end="")

test error 2068.83, train error 2031.65
test error 1867.44, train error 1847.32

In [ ]:
#Save the model
saver.save(sess, "./tf-models/cifar10.ckpt")

In [ ]:
image = random.randint(0,len(test_data))
num = 5

res = sess.run(result, feed_dict={x: test_data[image:image+num], keep_prob: 1})

for j,e in enumerate(res):
    data = []
    for i in range(32):
        tmp = []
        for k in range(32):
            img_dat_r = test[b'data'][image+j][32*i+k-1]
            img_dat_g = test[b'data'][image+j][1024+32*i+k-1]
            img_dat_b = test[b'data'][image+j][2048+32*i+k-1]
            tmp.append([img_dat_r,img_dat_g,img_dat_b])
        data.append(tmp)
    dat = np.array(data, dtype=np.float32)
    
    fig_size = plt.rcParams["figure.figsize"]
    fig_size[0] = 2
    fig_size[1] = 2
    plt.rcParams["figure.figsize"] = fig_size
    
    plt.imshow(dat)
    plt.show()
    
    print("suggested label: {:s}\nconfidence: {:.0f}%\ncorrect label: {:s}".format(cifar10_name[e.argmax()], e.max()*100, cifar10_name[test_labels[image+j].argmax()]))
    
    #for i in range(10):
    #    print(lables[i], " \t", e[i], " \t", test_labels[image+j][i])
    #print()